## Setting
- master1_cleansing_map_after.csv 을 이용해 추가 클린징 진행
- 위 파일 배포자 참고▼   https://github.com/eeuunnzzyy/eeuunnzzyy/blob/main/The%20Transportation%20Vulnerable%20Project/000%20Dailydata%20Cleansing%20%26%20EDA.ipynb

In [865]:
import pandas as pd
import numpy as np
import datetime 
import seaborn as sns
import glob

In [866]:
paths = glob.glob('./data/데이터전처리/*.csv')
paths

['./data/데이터전처리/전2_master1_cleansing_map_before.csv',
 './data/데이터전처리/전3_master_cleansing_before_diff.csv',
 './data/데이터전처리/master.csv',
 './data/데이터전처리/전4_master_cleansing_map_after.csv',
 './data/데이터전처리/전1_master_cleansing_space_time.csv']

In [867]:
paths[3]

'./data/데이터전처리/전4_master_cleansing_map_after.csv'

In [868]:
df = pd.read_csv(paths[3])
df.drop('Unnamed: 0', axis=1, inplace=True)
df.head()

,cartype,예정시간,배차시간,승차시간,startpos1,startpos2,endpos1,endpos2,start_lat,start_lng,end_lat,end_lng
0,중형승합,2018-01-23 00:04:00,2018-01-23 00:18:42,2018-01-23 00:34:08,성북구,장위제1동,동대문구,회기동,127.043713,37.614125,127.055313,37.590808
1,중형승합,2018-01-23 00:17:00,2018-01-23 00:50:37,2018-01-23 01:19:22,중랑구,면목제7동,노원구,상계1동,127.086985,37.578995,127.054978,37.679901
2,중형승합,2018-01-23 00:30:00,2018-01-23 00:17:06,2018-01-23 00:17:39,성북구,정릉제3동,강북구,수유제3동,127.004186,37.608958,127.023125,37.638715
3,중형승합,2018-01-23 00:47:00,2018-01-23 00:51:19,2018-01-23 01:09:33,구로구,고척제2동,관악구,은천동,126.858712,37.506706,126.942428,37.485309
4,중형승합,2018-01-23 01:08:00,2018-01-23 01:44:17,2018-01-23 02:06:53,동대문구,회기동,성북구,장위제2동,127.055313,37.590808,127.054521,37.612390


In [869]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 407553 entries, 0 to 407552
Data columns (total 12 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   cartype    407553 non-null  object 
 1   예정시간       407553 non-null  object 
 2   배차시간       407553 non-null  object 
 3   승차시간       407553 non-null  object 
 4   startpos1  407553 non-null  object 
 5   startpos2  407553 non-null  object 
 6   endpos1    407553 non-null  object 
 7   endpos2    407553 non-null  object 
 8   start_lat  407553 non-null  float64
 9   start_lng  407553 non-null  float64
 10  end_lat    407553 non-null  float64
 11  end_lng    407553 non-null  float64
dtypes: float64(4), object(8)
memory usage: 37.3+ MB


In [870]:
df.shape

(407553, 12)

## 시계열 적재오류 점검
- master1_cleansing_map_before.csv

- 시간데이터 체크list: (예상시간< 배차시간 < 승차시간) 
1. 위 날짜가 역순인경우 존재
2. 낮12시에서 낮 1시로 넘어갈때 13시로 넣어가지 않은 case -> 낮12시 00시로 변경하여 해겨
3. 배차시간 > 승차시간의 차이가 분단위가 아니라 일단위인 case -> 장애인콜택시측 : 적제오류

In [871]:
df['예정시간'].sort_values(ascending = False)

406845    2022-06-25 08:20:00
406844    2022-06-25 08:19:49
406843    2022-06-25 08:19:33
406842    2022-06-25 08:18:46
406841    2022-06-25 08:18:25
                 ...         
4484      2018-01-01 00:25:00
4483      2018-01-01 00:21:00
4482      2018-01-01 00:08:00
4481      2018-01-01 00:04:00
4480      2018-01-01 00:03:00
Name: 예정시간, Length: 407553, dtype: object

In [872]:
df['배차시간'].sort_values(ascending = False)

406845    2022-06-25 08:33:47
406844    2022-06-25 08:29:39
406841    2022-06-25 08:28:01
406842    2022-06-25 08:26:22
406843    2022-06-25 08:26:21
                 ...         
4488      2018-01-01 02:47:07
4487      2018-01-01 02:42:20
4482      2018-01-01 02:27:53
4481      2018-01-01 02:24:19
4480      2018-01-01 02:19:23
Name: 배차시간, Length: 407553, dtype: object

In [873]:
df['승차시간'].sort_values(ascending = False)

406815    2022-06-25 09:00:04
406811    2022-06-25 08:58:29
406813    2022-06-25 08:55:10
406812    2022-06-25 08:49:03
406814    2022-06-25 08:48:00
                 ...         
4487      2018-01-01 03:15:58
4488      2018-01-01 03:01:41
4480      2018-01-01 02:48:48
4482      2018-01-01 02:48:29
4481      2018-01-01 02:46:04
Name: 승차시간, Length: 407553, dtype: object

※ 낮12시 ->00시로 수정

In [874]:
# 여기부터 적재오류 test 할 때만 돌리기
df["예정시간"]=pd.to_datetime(df["예정시간"])
df["배차시간"]=pd.to_datetime(df["배차시간"])
df["승차시간"]=pd.to_datetime(df["승차시간"])

In [875]:
# 역순 점검 : 예정-배차_역순, 배차-승차_역순. Test1
역순 =[]

for i in range(len(df)):
    time_1 = df['예정시간'][i]
    time_2 = df['배차시간'][i]
    time_3 = df['승차시간'][i]
    
    if time_1 > time_2: # datetime 끼리 비교 가능  
        역순.append(1)
    elif time_2 > time_3: 
        역순.append(1)
    else:
        역순.append(0)
        

In [876]:
# 역순 점검: 배차-승차_역순. Test2
역순 =[]

for i in range(len(df)):
    time_2 = df['배차시간'][i]
    time_3 = df['승차시간'][i]
    
    if time_2 > time_3: 
        역순.append(1)
    else:
        역순.append(0)

In [877]:
df['역순'] = 역순

In [878]:
aa = df[df['역순']==1] # 배차시간>승차시간인 case : 407,553 중 51,405행 - 장애인 콜택시측에서도 적재오류로 설명함. 아예 제외할지 or 대체 값을 넣을지 결정하자!

In [879]:
aa

,cartype,예정시간,배차시간,승차시간,startpos1,startpos2,endpos1,endpos2,start_lat,start_lng,end_lat,end_lng,역순
1139,중형승합,2018-01-09 07:00:00,2018-01-09 07:05:24,2018-01-09 07:00:00,양천구,목1동,양천구,목5동,126.871232,37.530378,126.881621,37.537120,1
1716,중형승용,2018-01-19 07:03:00,2018-01-19 07:18:40,2018-01-19 07:03:00,도봉구,창제1동,도봉구,창제1동,127.043868,37.648305,127.043868,37.648305,1
3738,중형승합,2018-01-02 03:24:00,2018-01-02 03:26:15,2018-01-02 03:24:00,노원구,상계6.7동,노원구,중계4동,127.066954,37.654882,127.078009,37.658727,1
3952,중형승합,2018-01-02 07:08:00,2018-01-02 07:11:36,2018-01-02 07:08:00,도봉구,쌍문제1동,강북구,인수동,127.026115,37.648027,127.010631,37.641511,1
4398,중형승합,2018-01-17 07:00:00,2018-01-17 07:04:20,2018-01-17 07:00:00,구로구,구로제3동,영등포구,대림제2동,126.890368,37.487380,126.898294,37.492752,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
407525,중형승합,2022-06-24 07:00:00,2022-06-24 07:30:33,2022-06-24 07:15:13,종로구,명륜3가동,동대문구,회기동,126.993100,37.587500,127.055313,37.590808,1
407536,중형승용,2022-06-24 07:00:00,2022-06-24 07:52:58,2022-06-24 07:32:45,강동구,상일동,마포구,상암동,127.168301,37.550583,126.894688,37.578325,1
407537,대형승용,2022-06-24 07:00:00,2022-06-24 08:04:45,2022-06-24 07:38:17,동대문구,장안제4동,은평구,구산동,127.070600,37.578483,126.910389,37.611745,1
407546,중형승합,2022-06-24 07:00:00,2022-06-24 07:01:18,2022-06-24 06:35:15,용산구,후암동,분당구,서현1동,126.978134,37.548637,127.126029,37.382926,1


In [880]:
aa.to_csv('master2_역순점검_.csv', index = False, encoding = 'utf-8')

In [881]:
# ------ 이상치 제거 후 -------

In [882]:
aa2 = df[df['역순']==0]

In [883]:
aa2 = aa2.drop('역순', axis=1)

In [884]:
aa2 # 407,553 -> 356,149 (51,404행 이상치 제거)

,cartype,예정시간,배차시간,승차시간,startpos1,startpos2,endpos1,endpos2,start_lat,start_lng,end_lat,end_lng
0,중형승합,2018-01-23 00:04:00,2018-01-23 00:18:42,2018-01-23 00:34:08,성북구,장위제1동,동대문구,회기동,127.043713,37.614125,127.055313,37.590808
1,중형승합,2018-01-23 00:17:00,2018-01-23 00:50:37,2018-01-23 01:19:22,중랑구,면목제7동,노원구,상계1동,127.086985,37.578995,127.054978,37.679901
2,중형승합,2018-01-23 00:30:00,2018-01-23 00:17:06,2018-01-23 00:17:39,성북구,정릉제3동,강북구,수유제3동,127.004186,37.608958,127.023125,37.638715
3,중형승합,2018-01-23 00:47:00,2018-01-23 00:51:19,2018-01-23 01:09:33,구로구,고척제2동,관악구,은천동,126.858712,37.506706,126.942428,37.485309
4,중형승합,2018-01-23 01:08:00,2018-01-23 01:44:17,2018-01-23 02:06:53,동대문구,회기동,성북구,장위제2동,127.055313,37.590808,127.054521,37.612390
...,...,...,...,...,...,...,...,...,...,...,...,...
407547,중형승합,2022-06-24 07:00:00,2022-06-24 07:09:37,2022-06-24 07:48:41,서대문구,홍제제2동,은평구,갈현제2동,126.949338,37.586107,126.915839,37.618587
407548,대형승용,2022-06-24 07:00:00,2022-06-24 07:12:38,2022-06-24 07:55:24,성북구,돈암제2동,강남구,대치4동,127.010566,37.598039,127.057913,37.499741
407549,중형승합,2022-06-24 07:00:00,2022-06-24 07:09:10,2022-06-24 07:25:35,은평구,구산동,은평구,구산동,126.910389,37.611745,126.910389,37.611745
407550,중형승합,2022-06-24 07:00:00,2022-06-24 07:04:27,2022-06-24 07:25:35,노원구,상계8동,영등포구,영등포동,127.051587,37.666785,126.910692,37.520398


In [885]:
aa2.to_csv('master2_역순점검_이상치제거.csv', index = False, encoding = 'utf-8')

In [886]:
df = pd.read_csv('./master2_역순점검_이상치제거.csv')

In [887]:
df.shape

(356149, 12)

## 1. 3가지 차이 시간 칼럼 생성

In [888]:
import time
import datetime

### '배차-예정'

In [889]:
%%time

배차예정_val = []
for i in range(len(df)):
    time_1 = datetime.datetime.strptime(df['예정시간'][i],"%Y-%m-%d %H:%M:%S")
    time_2 = datetime.datetime.strptime(df['배차시간'][i],"%Y-%m-%d %H:%M:%S")
    time_interval = time_2 - time_1
    배차예정_val.append(time_interval)
    
##날짜와 시간(datetime)을 문자열로 출력하려면 strftime
##날짜와 시간 형식의 문자열을 datetime으로 변환하려면 strptime을 사용하면 된다

CPU times: user 7.22 s, sys: 6.13 ms, total: 7.23 s
Wall time: 7.23 s


In [890]:
df['배차-예정'] = 배차예정_val

### '승차-배차'

In [891]:
%%time

승차배차_val = []
for i in range(len(df)):
    time_1 = datetime.datetime.strptime(df['배차시간'][i],"%Y-%m-%d %H:%M:%S")
    time_2 = datetime.datetime.strptime(df['승차시간'][i],"%Y-%m-%d %H:%M:%S")
    time_interval = time_2 - time_1
    승차배차_val.append(time_interval)

CPU times: user 7.21 s, sys: 9.2 ms, total: 7.22 s
Wall time: 7.22 s


In [892]:
df['승차-배차'] = 승차배차_val

### '승차-예정'

In [893]:
%%time

승차예정_val = []
for i in range(len(df)):
    time_1 = datetime.datetime.strptime(df['예정시간'][i],"%Y-%m-%d %H:%M:%S")
    time_2 = datetime.datetime.strptime(df['승차시간'][i],"%Y-%m-%d %H:%M:%S")
    time_interval = time_2 - time_1
    승차예정_val.append(time_interval)

CPU times: user 7.21 s, sys: 9.91 ms, total: 7.22 s
Wall time: 7.22 s


In [894]:
df['승차-예정'] = 승차예정_val

In [895]:
df.head()

,cartype,예정시간,배차시간,승차시간,startpos1,startpos2,endpos1,endpos2,start_lat,start_lng,end_lat,end_lng,배차-예정,승차-배차,승차-예정
0,중형승합,2018-01-23 00:04:00,2018-01-23 00:18:42,2018-01-23 00:34:08,성북구,장위제1동,동대문구,회기동,127.043713,37.614125,127.055313,37.590808,0 days 00:14:42,0 days 00:15:26,0 days 00:30:08
1,중형승합,2018-01-23 00:17:00,2018-01-23 00:50:37,2018-01-23 01:19:22,중랑구,면목제7동,노원구,상계1동,127.086985,37.578995,127.054978,37.679901,0 days 00:33:37,0 days 00:28:45,0 days 01:02:22
2,중형승합,2018-01-23 00:30:00,2018-01-23 00:17:06,2018-01-23 00:17:39,성북구,정릉제3동,강북구,수유제3동,127.004186,37.608958,127.023125,37.638715,-1 days +23:47:06,0 days 00:00:33,-1 days +23:47:39
3,중형승합,2018-01-23 00:47:00,2018-01-23 00:51:19,2018-01-23 01:09:33,구로구,고척제2동,관악구,은천동,126.858712,37.506706,126.942428,37.485309,0 days 00:04:19,0 days 00:18:14,0 days 00:22:33
4,중형승합,2018-01-23 01:08:00,2018-01-23 01:44:17,2018-01-23 02:06:53,동대문구,회기동,성북구,장위제2동,127.055313,37.590808,127.054521,37.612390,0 days 00:36:17,0 days 00:22:36,0 days 00:58:53


예상시간보다 일찍 도착하면, 시간차이 구할때 -1day씩 추가계산되는 문제가 있다.

### -1 days 점검

In [896]:
%%time

배차예정2 = []
for i in range(len(df)):
    배차예정2.append(str(df['배차-예정'][i]))

CPU times: user 4.12 s, sys: 23 ms, total: 4.14 s
Wall time: 4.14 s


In [897]:
df['배차-예정test']=배차예정2

In [898]:
df[df['배차-예정test'].str.contains('-1 days')] ## 356,149 중 34,261행 존재 -> 약 9.6% 일찍 배차

,cartype,예정시간,배차시간,승차시간,startpos1,startpos2,endpos1,endpos2,start_lat,start_lng,end_lat,end_lng,배차-예정,승차-배차,승차-예정,배차-예정test
2,중형승합,2018-01-23 00:30:00,2018-01-23 00:17:06,2018-01-23 00:17:39,성북구,정릉제3동,강북구,수유제3동,127.004186,37.608958,127.023125,37.638715,-1 days +23:47:06,0 days 00:00:33,-1 days +23:47:39,-1 days +23:47:06
25,중형승합,2018-01-23 04:30:00,2018-01-23 04:00:59,2018-01-23 04:47:02,구로구,오류제1동,영등포구,대림제3동,126.845201,37.497009,126.898031,37.498335,-1 days +23:30:59,0 days 00:46:03,0 days 00:17:02,-1 days +23:30:59
27,중형승합,2018-01-23 04:40:00,2018-01-23 04:02:59,2018-01-23 04:43:09,종로구,부암동,종로구,창신제1동,126.964100,37.592478,127.016363,37.570758,-1 days +23:22:59,0 days 00:40:10,0 days 00:03:09,-1 days +23:22:59
28,중형승합,2018-01-23 04:40:00,2018-01-23 03:57:30,2018-01-23 04:44:27,송파구,장지동,강동구,명일제1동,127.135400,37.478500,127.144366,37.551245,-1 days +23:17:30,0 days 00:46:57,0 days 00:04:27,-1 days +23:17:30
30,중형승합,2018-01-23 05:00:00,2018-01-23 04:32:36,2018-01-23 04:56:19,성북구,종암동,종로구,청운효자동,127.035157,37.599007,126.970652,37.584137,-1 days +23:32:36,0 days 00:23:43,-1 days +23:56:19,-1 days +23:32:36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
356111,중형승용,2022-06-24 07:00:00,2022-06-24 06:35:46,2022-06-24 08:12:29,성동구,마장동,서초구,서초1동,127.045388,37.566325,127.019508,37.490119,-1 days +23:35:46,0 days 01:36:43,0 days 01:12:29,-1 days +23:35:46
356114,중형승합,2022-06-24 07:00:00,2022-06-24 06:19:36,2022-06-24 07:25:45,관악구,낙성대동,서초구,반포4동,126.958388,37.476297,127.000385,37.497431,-1 days +23:19:36,0 days 01:06:09,0 days 00:25:45,-1 days +23:19:36
356115,중형승합,2022-06-24 07:00:00,2022-06-24 06:40:43,2022-06-24 07:20:49,강서구,화곡제4동,구로구,수궁동,126.860907,37.534648,126.831463,37.493942,-1 days +23:40:43,0 days 00:40:06,0 days 00:20:49,-1 days +23:40:43
356125,중형승용,2022-06-24 07:00:00,2022-06-24 06:28:40,2022-06-24 07:30:24,강서구,화곡본동,강남구,청담동,126.847725,37.544075,127.049291,37.525107,-1 days +23:28:40,0 days 01:01:44,0 days 00:30:24,-1 days +23:28:40


-----------------------

In [899]:
# for문전에 돌리기
df["예정시간"]=pd.to_datetime(df["예정시간"])
df["배차시간"]=pd.to_datetime(df["배차시간"])
df["승차시간"]=pd.to_datetime(df["승차시간"])

In [900]:
# test1

배차예정_val = []
for i in range(len(df)):
    time_1 = df['예정시간'][i]
    time_2 = df['배차시간'][i]
    
    if time_2 < time_1: # datetime 끼리 비교 가능  
        time_3 = time_2 + datetime.timedelta(hours=23, minutes=60, seconds=60) # datetime + timedelta 가능
        time_interval = time_3 - time_1
        배차예정_val.append(time_interval)
    else:
        time_interval = time_2 - time_1
        배차예정_val.append(time_interval)

In [901]:
# test2 채택★- 순시간구하기 -> 일찍 도착한 셀 마이너스 붙이기 전!

배차예정_val = []
for i in range(len(df)):
    time_1 = df['예정시간'][i]
    time_2 = df['배차시간'][i]
    
    if time_2 < time_1: # datetime 끼리 비교 가능  
        time_interval = time_1 - time_2
        배차예정_val.append(time_interval)
    else:
        time_interval = time_2 - time_1
        배차예정_val.append(time_interval)

승차배차_val = []
for i in range(len(df)):
    time_1 = df['배차시간'][i]
    time_2 = df['승차시간'][i]
    
    if time_2 < time_1: # datetime 끼리 비교 가능  
        time_interval = time_1 - time_2
        승차배차_val.append(time_interval)
    else:
        time_interval = time_2 - time_1
        승차배차_val.append(time_interval)
        
승차예정_val = []
for i in range(len(df)):
    time_1 = df['예정시간'][i]
    time_2 = df['승차시간'][i]
    
    if time_2 < time_1: # datetime 끼리 비교 가능  
        time_interval = time_1 - time_2
        승차예정_val.append(time_interval)
    else:
        time_interval = time_2 - time_1
        승차예정_val.append(time_interval)

In [902]:
df['배차-예정'] = 배차예정_val 
df['승차-배차'] = 승차배차_val 
df['승차-예정'] = 승차예정_val 

In [903]:
df

,cartype,예정시간,배차시간,승차시간,startpos1,startpos2,endpos1,endpos2,start_lat,start_lng,end_lat,end_lng,배차-예정,승차-배차,승차-예정,배차-예정test
0,중형승합,2018-01-23 00:04:00,2018-01-23 00:18:42,2018-01-23 00:34:08,성북구,장위제1동,동대문구,회기동,127.043713,37.614125,127.055313,37.590808,0 days 00:14:42,0 days 00:15:26,0 days 00:30:08,0 days 00:14:42
1,중형승합,2018-01-23 00:17:00,2018-01-23 00:50:37,2018-01-23 01:19:22,중랑구,면목제7동,노원구,상계1동,127.086985,37.578995,127.054978,37.679901,0 days 00:33:37,0 days 00:28:45,0 days 01:02:22,0 days 00:33:37
2,중형승합,2018-01-23 00:30:00,2018-01-23 00:17:06,2018-01-23 00:17:39,성북구,정릉제3동,강북구,수유제3동,127.004186,37.608958,127.023125,37.638715,0 days 00:12:54,0 days 00:00:33,0 days 00:12:21,-1 days +23:47:06
3,중형승합,2018-01-23 00:47:00,2018-01-23 00:51:19,2018-01-23 01:09:33,구로구,고척제2동,관악구,은천동,126.858712,37.506706,126.942428,37.485309,0 days 00:04:19,0 days 00:18:14,0 days 00:22:33,0 days 00:04:19
4,중형승합,2018-01-23 01:08:00,2018-01-23 01:44:17,2018-01-23 02:06:53,동대문구,회기동,성북구,장위제2동,127.055313,37.590808,127.054521,37.612390,0 days 00:36:17,0 days 00:22:36,0 days 00:58:53,0 days 00:36:17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
356144,중형승합,2022-06-24 07:00:00,2022-06-24 07:09:37,2022-06-24 07:48:41,서대문구,홍제제2동,은평구,갈현제2동,126.949338,37.586107,126.915839,37.618587,0 days 00:09:37,0 days 00:39:04,0 days 00:48:41,0 days 00:09:37
356145,대형승용,2022-06-24 07:00:00,2022-06-24 07:12:38,2022-06-24 07:55:24,성북구,돈암제2동,강남구,대치4동,127.010566,37.598039,127.057913,37.499741,0 days 00:12:38,0 days 00:42:46,0 days 00:55:24,0 days 00:12:38
356146,중형승합,2022-06-24 07:00:00,2022-06-24 07:09:10,2022-06-24 07:25:35,은평구,구산동,은평구,구산동,126.910389,37.611745,126.910389,37.611745,0 days 00:09:10,0 days 00:16:25,0 days 00:25:35,0 days 00:09:10
356147,중형승합,2022-06-24 07:00:00,2022-06-24 07:04:27,2022-06-24 07:25:35,노원구,상계8동,영등포구,영등포동,127.051587,37.666785,126.910692,37.520398,0 days 00:04:27,0 days 00:21:08,0 days 00:25:35,0 days 00:04:27


0days 00:00:00 형식에서 분단위에 마이너스 붙이는 방법 누가알려줘라 

### 시간차이 칼럼 분단위 통합

In [904]:
#test- 분변환
duration = datetime.timedelta(days = 0, hours=2, minutes=30, seconds=60)
seconds = duration.total_seconds()
minutes = seconds//60

minutes

151.0

In [905]:
# 반복 
배차예정_val = []

for i in range(len(df)):
    time_1 = df['예정시간'][i]
    time_2 = df['배차시간'][i]
    
    if time_2 < time_1: # datetime 끼리 비교 가능  
        time_interval = time_1 - time_2
        
        seconds = time_interval.total_seconds()
        minutes = -seconds//60

        배차예정_val.append(minutes)
    else:
        time_interval = time_2 - time_1
        
        seconds = time_interval.total_seconds()
        minutes = seconds//60
        
        배차예정_val.append(minutes)

In [906]:
df['배차-예정(분)'] = 배차예정_val

In [907]:
df

,cartype,예정시간,배차시간,승차시간,startpos1,startpos2,endpos1,endpos2,start_lat,start_lng,end_lat,end_lng,배차-예정,승차-배차,승차-예정,배차-예정test,배차-예정(분)
0,중형승합,2018-01-23 00:04:00,2018-01-23 00:18:42,2018-01-23 00:34:08,성북구,장위제1동,동대문구,회기동,127.043713,37.614125,127.055313,37.590808,0 days 00:14:42,0 days 00:15:26,0 days 00:30:08,0 days 00:14:42,14.0
1,중형승합,2018-01-23 00:17:00,2018-01-23 00:50:37,2018-01-23 01:19:22,중랑구,면목제7동,노원구,상계1동,127.086985,37.578995,127.054978,37.679901,0 days 00:33:37,0 days 00:28:45,0 days 01:02:22,0 days 00:33:37,33.0
2,중형승합,2018-01-23 00:30:00,2018-01-23 00:17:06,2018-01-23 00:17:39,성북구,정릉제3동,강북구,수유제3동,127.004186,37.608958,127.023125,37.638715,0 days 00:12:54,0 days 00:00:33,0 days 00:12:21,-1 days +23:47:06,-13.0
3,중형승합,2018-01-23 00:47:00,2018-01-23 00:51:19,2018-01-23 01:09:33,구로구,고척제2동,관악구,은천동,126.858712,37.506706,126.942428,37.485309,0 days 00:04:19,0 days 00:18:14,0 days 00:22:33,0 days 00:04:19,4.0
4,중형승합,2018-01-23 01:08:00,2018-01-23 01:44:17,2018-01-23 02:06:53,동대문구,회기동,성북구,장위제2동,127.055313,37.590808,127.054521,37.612390,0 days 00:36:17,0 days 00:22:36,0 days 00:58:53,0 days 00:36:17,36.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
356144,중형승합,2022-06-24 07:00:00,2022-06-24 07:09:37,2022-06-24 07:48:41,서대문구,홍제제2동,은평구,갈현제2동,126.949338,37.586107,126.915839,37.618587,0 days 00:09:37,0 days 00:39:04,0 days 00:48:41,0 days 00:09:37,9.0
356145,대형승용,2022-06-24 07:00:00,2022-06-24 07:12:38,2022-06-24 07:55:24,성북구,돈암제2동,강남구,대치4동,127.010566,37.598039,127.057913,37.499741,0 days 00:12:38,0 days 00:42:46,0 days 00:55:24,0 days 00:12:38,12.0
356146,중형승합,2022-06-24 07:00:00,2022-06-24 07:09:10,2022-06-24 07:25:35,은평구,구산동,은평구,구산동,126.910389,37.611745,126.910389,37.611745,0 days 00:09:10,0 days 00:16:25,0 days 00:25:35,0 days 00:09:10,9.0
356147,중형승합,2022-06-24 07:00:00,2022-06-24 07:04:27,2022-06-24 07:25:35,노원구,상계8동,영등포구,영등포동,127.051587,37.666785,126.910692,37.520398,0 days 00:04:27,0 days 00:21:08,0 days 00:25:35,0 days 00:04:27,4.0


'배차-예정' 마이너스 값 + 분단위 통합 성공 -> 이제 나머지 '승차-배차', '승차-예정'도 통합시켜주자

In [908]:
%%time

승차배차_val = []

for i in range(len(df)):
    time_1 = df['배차시간'][i]
    time_2 = df['승차시간'][i]
    
    if time_2 < time_1: # datetime 끼리 비교 가능  
        time_interval = time_1 - time_2
        
        seconds = time_interval.total_seconds()
        minutes = -seconds//60

        승차배차_val.append(minutes)
    else:
        time_interval = time_2 - time_1
        
        seconds = time_interval.total_seconds()
        minutes = seconds//60
        
        승차배차_val.append(minutes)


CPU times: user 7.93 s, sys: 12.6 ms, total: 7.95 s
Wall time: 7.95 s


In [909]:
df['승차-배차(분)'] = 승차배차_val

In [910]:
%%time

승차예정_val = []

for i in range(len(df)):
    time_1 = df['예정시간'][i]
    time_2 = df['승차시간'][i]
    
    if time_2 < time_1: # datetime 끼리 비교 가능  
        time_interval = time_1 - time_2
        
        seconds = time_interval.total_seconds()
        minutes = -seconds//60

        승차예정_val.append(minutes)
    else:
        time_interval = time_2 - time_1
        
        seconds = time_interval.total_seconds()
        minutes = seconds//60
        
        승차예정_val.append(minutes)

CPU times: user 7.87 s, sys: 8.65 ms, total: 7.88 s
Wall time: 7.88 s


In [911]:
df['승차-예정(분)'] = 승차예정_val

In [912]:
df

,cartype,예정시간,배차시간,승차시간,startpos1,startpos2,endpos1,endpos2,start_lat,start_lng,end_lat,end_lng,배차-예정,승차-배차,승차-예정,배차-예정test,배차-예정(분),승차-배차(분),승차-예정(분)
0,중형승합,2018-01-23 00:04:00,2018-01-23 00:18:42,2018-01-23 00:34:08,성북구,장위제1동,동대문구,회기동,127.043713,37.614125,127.055313,37.590808,0 days 00:14:42,0 days 00:15:26,0 days 00:30:08,0 days 00:14:42,14.0,15.0,30.0
1,중형승합,2018-01-23 00:17:00,2018-01-23 00:50:37,2018-01-23 01:19:22,중랑구,면목제7동,노원구,상계1동,127.086985,37.578995,127.054978,37.679901,0 days 00:33:37,0 days 00:28:45,0 days 01:02:22,0 days 00:33:37,33.0,28.0,62.0
2,중형승합,2018-01-23 00:30:00,2018-01-23 00:17:06,2018-01-23 00:17:39,성북구,정릉제3동,강북구,수유제3동,127.004186,37.608958,127.023125,37.638715,0 days 00:12:54,0 days 00:00:33,0 days 00:12:21,-1 days +23:47:06,-13.0,0.0,-13.0
3,중형승합,2018-01-23 00:47:00,2018-01-23 00:51:19,2018-01-23 01:09:33,구로구,고척제2동,관악구,은천동,126.858712,37.506706,126.942428,37.485309,0 days 00:04:19,0 days 00:18:14,0 days 00:22:33,0 days 00:04:19,4.0,18.0,22.0
4,중형승합,2018-01-23 01:08:00,2018-01-23 01:44:17,2018-01-23 02:06:53,동대문구,회기동,성북구,장위제2동,127.055313,37.590808,127.054521,37.612390,0 days 00:36:17,0 days 00:22:36,0 days 00:58:53,0 days 00:36:17,36.0,22.0,58.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
356144,중형승합,2022-06-24 07:00:00,2022-06-24 07:09:37,2022-06-24 07:48:41,서대문구,홍제제2동,은평구,갈현제2동,126.949338,37.586107,126.915839,37.618587,0 days 00:09:37,0 days 00:39:04,0 days 00:48:41,0 days 00:09:37,9.0,39.0,48.0
356145,대형승용,2022-06-24 07:00:00,2022-06-24 07:12:38,2022-06-24 07:55:24,성북구,돈암제2동,강남구,대치4동,127.010566,37.598039,127.057913,37.499741,0 days 00:12:38,0 days 00:42:46,0 days 00:55:24,0 days 00:12:38,12.0,42.0,55.0
356146,중형승합,2022-06-24 07:00:00,2022-06-24 07:09:10,2022-06-24 07:25:35,은평구,구산동,은평구,구산동,126.910389,37.611745,126.910389,37.611745,0 days 00:09:10,0 days 00:16:25,0 days 00:25:35,0 days 00:09:10,9.0,16.0,25.0
356147,중형승합,2022-06-24 07:00:00,2022-06-24 07:04:27,2022-06-24 07:25:35,노원구,상계8동,영등포구,영등포동,127.051587,37.666785,126.910692,37.520398,0 days 00:04:27,0 days 00:21:08,0 days 00:25:35,0 days 00:04:27,4.0,21.0,25.0


In [913]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 356149 entries, 0 to 356148
Data columns (total 19 columns):
 #   Column     Non-Null Count   Dtype          
---  ------     --------------   -----          
 0   cartype    356149 non-null  object         
 1   예정시간       356149 non-null  datetime64[ns] 
 2   배차시간       356149 non-null  datetime64[ns] 
 3   승차시간       356149 non-null  datetime64[ns] 
 4   startpos1  356149 non-null  object         
 5   startpos2  356149 non-null  object         
 6   endpos1    356149 non-null  object         
 7   endpos2    356149 non-null  object         
 8   start_lat  356149 non-null  float64        
 9   start_lng  356149 non-null  float64        
 10  end_lat    356149 non-null  float64        
 11  end_lng    356149 non-null  float64        
 12  배차-예정      356149 non-null  timedelta64[ns]
 13  승차-배차      356149 non-null  timedelta64[ns]
 14  승차-예정      356149 non-null  timedelta64[ns]
 15  배차-예정test  356149 non-null  object         
 16  배차

In [914]:
df.drop(['배차-예정','승차-배차','승차-예정','배차-예정test'], axis=1, inplace=True)

In [915]:
df

,cartype,예정시간,배차시간,승차시간,startpos1,startpos2,endpos1,endpos2,start_lat,start_lng,end_lat,end_lng,배차-예정(분),승차-배차(분),승차-예정(분)
0,중형승합,2018-01-23 00:04:00,2018-01-23 00:18:42,2018-01-23 00:34:08,성북구,장위제1동,동대문구,회기동,127.043713,37.614125,127.055313,37.590808,14.0,15.0,30.0
1,중형승합,2018-01-23 00:17:00,2018-01-23 00:50:37,2018-01-23 01:19:22,중랑구,면목제7동,노원구,상계1동,127.086985,37.578995,127.054978,37.679901,33.0,28.0,62.0
2,중형승합,2018-01-23 00:30:00,2018-01-23 00:17:06,2018-01-23 00:17:39,성북구,정릉제3동,강북구,수유제3동,127.004186,37.608958,127.023125,37.638715,-13.0,0.0,-13.0
3,중형승합,2018-01-23 00:47:00,2018-01-23 00:51:19,2018-01-23 01:09:33,구로구,고척제2동,관악구,은천동,126.858712,37.506706,126.942428,37.485309,4.0,18.0,22.0
4,중형승합,2018-01-23 01:08:00,2018-01-23 01:44:17,2018-01-23 02:06:53,동대문구,회기동,성북구,장위제2동,127.055313,37.590808,127.054521,37.612390,36.0,22.0,58.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
356144,중형승합,2022-06-24 07:00:00,2022-06-24 07:09:37,2022-06-24 07:48:41,서대문구,홍제제2동,은평구,갈현제2동,126.949338,37.586107,126.915839,37.618587,9.0,39.0,48.0
356145,대형승용,2022-06-24 07:00:00,2022-06-24 07:12:38,2022-06-24 07:55:24,성북구,돈암제2동,강남구,대치4동,127.010566,37.598039,127.057913,37.499741,12.0,42.0,55.0
356146,중형승합,2022-06-24 07:00:00,2022-06-24 07:09:10,2022-06-24 07:25:35,은평구,구산동,은평구,구산동,126.910389,37.611745,126.910389,37.611745,9.0,16.0,25.0
356147,중형승합,2022-06-24 07:00:00,2022-06-24 07:04:27,2022-06-24 07:25:35,노원구,상계8동,영등포구,영등포동,127.051587,37.666785,126.910692,37.520398,4.0,21.0,25.0


In [917]:
df.to_csv('master_cleansing_diff_after.csv', index = True, encoding = 'utf-8')